In [1]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# Processing
from sklearn.preprocessing import StandardScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Functions and definitions

In [2]:
def score_model(model, x, y, n_splits=10, n_repeats=10):
    scoring = "neg_root_mean_squared_error"
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=SEED)
    scores = cross_val_score(model, x, y, scoring=scoring, cv=cv, n_jobs=-1)
    return scores

In [3]:
def plot_predictions(linreg_model, df, index, x, y):
    d = df["Data/Hora"].values
    test_series = pd.DataFrame({"ccs28": y}, index=pd.to_datetime(d))
    pred_series = linreg_model.predict(scaler.transform(x))
    pred_series = pd.DataFrame({"ccs28-pred": pred_series}, index=pd.to_datetime(d))

    fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(15, 7))

    test_series.plot(ax=ax)
    ax.axvline(test_series.index[index], color="r")  # end of train dataset
    pred_series[index:].plot(ax=ax)
    ax.grid(which="both")
    ax.legend(
        ["train and test series", "end of train series", "predicted"], loc="upper left"
    )
    # ax.set_xlabel('Period', labelpad=20, fontsize=15)
    ax.set_ylabel("Compressive Strength - MPa", labelpad=20, fontsize=15)
    plt.show()

In [4]:
def plot_scores_box_plot(scores, repeats, n_splits):
    plt.figure(figsize=(15, 8))
    plt.boxplot(
        scores.reshape((repeats, n_splits)),
        labels=[str(r) for r in range(1, repeats + 1)],
        showmeans=True,
    )
    plt.ylabel("RMSE", labelpad=20, fontsize=15)
    plt.xlabel("Repeats", labelpad=20, fontsize=15)
    plt.show()

In [5]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.5 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

# Reference: https://goldinlocks.github.io/Time-Series-Cross-Validation/

In [6]:
def repeated_blocking_time_series(
    Estimator,
    Transform,
    x,
    y,
    train_period,
    test_period,
    repeats=10,
    params=None
):
    results = []
    max_samples = x.shape[0]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // (train_period + test_period)):
            scaler = Transform()
            model = Estimator() if params is None else Estimator(**params)
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            x_train = scaler.fit_transform(x_train)
            x_test = scaler.transform(x_test)

            model.fit(x_train, y_train)

            y_train_pred = model.predict(x_train)
            y_test_pred = model.predict(x_test)
            scores.append(
                score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
            )
            i += train_period + test_period
        
        scores_final = {key: [] for key, _ in scores[0].items()}
        for scores_dict in scores:
            for key, value in scores_dict.items():
                scores_final[key] += [value]
        results.append(scores_final)
    return results

In [7]:
def print_scores(scores, METRICS, METRICS_DICT):
    for phase in ["train", "test"]:
        print("******")
        print(f"[{phase.upper()}]")
        print("******")
        for metric in METRICS:
            name = METRICS_DICT[metric]           
            print(
                f"{name}: %.3f (%.3f)"
                % (
                    np.abs(np.mean(scores[f"{phase}_" + metric])),
                    np.std(scores[f"{phase}_" + metric]),
                )
            )
        print("\n======================\n")

In [8]:
def score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred):
    TRAIN_RMSE = mean_squared_error(y_true=y_train, y_pred=y_train_pred, squared=False)
    TRAIN_MAE = mean_absolute_error(y_true=y_train, y_pred=y_train_pred)
    TRAIN_MAPE = mean_absolute_percentage_error(y_true=y_train, y_pred=y_train_pred)
    TRAIN_R2 = r2_score(y_true=y_train, y_pred=y_train_pred)

    TEST_RMSE = mean_squared_error(y_true=y_test, y_pred=y_test_pred, squared=False)
    TEST_MAE = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
    TEST_MAPE = mean_absolute_percentage_error(y_true=y_test, y_pred=y_test_pred)
    TEST_R2 = r2_score(y_true=y_test, y_pred=y_test_pred)

    scores = {
        "train_neg_root_mean_squared_error": TRAIN_RMSE,
        "train_neg_mean_absolute_error": TRAIN_MAE,
        "train_neg_mean_absolute_percentage_error": TRAIN_MAPE,
        "train_r2": TRAIN_R2,
        "test_neg_root_mean_squared_error": TEST_RMSE,
        "test_neg_mean_absolute_error": TEST_MAE,
        "test_neg_mean_absolute_percentage_error": TEST_MAPE,
        "test_r2": TEST_R2,
    }
    return scores

In [9]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

# Reading the dataset

In [10]:
df = pd.read_csv("../../../../../data/processed/partner_i-Oficial/cpiif40.csv")

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered.

In [11]:
df_no_cs = df.drop(
    [
        "1 day Compressive strength",
        "3 day Compressive strength",
        "7 day Compressive strength",
    ],
    axis=1,
).copy()

y = df_no_cs.pop('28 day Compressive strength').values
x = df_no_cs.drop(['Date'], axis=1)

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (594, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 59 samples each
    2. 90% train (535 samples each fold)
    3. 10% test (59 samples each fold)
<b>Total:</b> 100 models<br>

In [12]:
repeats = 10
n_splits = 10
pipeline = Pipeline([("transformer", StandardScaler()), ("estimator", LinearRegression())])
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
# scores = np.abs(scores)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

Repeated Cross Validation:
Repeats: 10
n_splits: 10

******
[TRAIN]
******
RMSE: 1.001 (0.012)
MAE: 0.784 (0.010)
MAPE: 0.016 (0.000)
R2: 0.281 (0.013)


******
[TEST]
******
RMSE: 10.464 (28.357)
MAE: 2.082 (3.718)
MAPE: 0.044 (0.079)
R2: 673.991 (2159.315)




## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (594, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 118 samples
    2. 50% train (59 samples each fold)
    3. 50% test (59 samples each fold)
<b>Total:</b> 5 models<br>

In [13]:
n_splits = 5
pipeline = Pipeline([("transformer", StandardScaler()), ("estimator", LinearRegression())])
cv = BlockingTimeSeriesSplit(n_splits=n_splits)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

Blocking Time Series Split:
Repeats: 10
n_splits: 5

******
[TRAIN]
******
RMSE: 0.498 (0.062)
MAE: 0.397 (0.058)
MAPE: 0.008 (0.001)
R2: 0.756 (0.080)


******
[TEST]
******
RMSE: 181.684 (265.675)
MAE: 25.536 (35.217)
MAPE: 0.518 (0.702)
R2: 57006.084 (71655.555)




## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (594, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 54, 108, 162, 216, 270, 324, 378, 432, 486 samples each fold
    2. Test: 54 samples each fold
<b>Total:</b> 10 models<br>

In [14]:
n_splits = 10
gap = 0
pipeline = Pipeline([("transformer", StandardScaler()), ("estimator", LinearRegression())])
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
# scores = np.abs(scores)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

Blocking Time Series Split:
Repeats: 10
n_splits: 10

******
[TRAIN]
******
RMSE: 0.786 (0.145)
MAE: 0.620 (0.109)
MAPE: 0.013 (0.002)
R2: 0.459 (0.176)


******
[TEST]
******
RMSE: 11.324 (25.512)
MAE: 2.593 (3.393)
MAPE: 0.055 (0.072)
R2: 1698.966 (5036.248)




## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (594, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 475
    2. Test: 118
<b>Total:</b> 1 model<br>

In [15]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [("transformer", StandardScaler()), ("estimator", LinearRegression())]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.916 (0.000)
MAE: 0.728 (0.000)
MAPE: 0.015 (0.000)
R2: 0.363 (0.000)


******
[TEST]
******
RMSE: 1.444 (0.000)
MAE: 1.190 (0.000)
MAPE: 0.025 (0.000)
R2: 0.247 (0.000)




## 1.5 Blocking Time Series - Period Specific

### 1.5.1 Train 6 months and predicts one month every year

In [16]:
scores = repeated_blocking_time_series(
    LinearRegression,
    StandardScaler,
    x,
    y,
    train_period=180,
    test_period=30,
    repeats=1,
)
print_scores(scores[0], METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.739 (0.062)
MAE: 0.578 (0.046)
MAPE: 0.012 (0.001)
R2: 0.403 (0.073)


******
[TEST]
******
RMSE: 0.763 (0.094)
MAE: 0.619 (0.050)
MAPE: 0.013 (0.001)
R2: 0.245 (0.091)




### 1.5.2 Train 11 months and predicts one month every year

In [17]:
scores = repeated_blocking_time_series(
    LinearRegression,
    StandardScaler,
    x,
    y,
    train_period=330,
    test_period=30,
    repeats=1,
)
print_scores(scores[0], METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.816 (0.000)
MAE: 0.638 (0.000)
MAPE: 0.013 (0.000)
R2: 0.333 (0.000)


******
[TEST]
******
RMSE: 116.114 (0.000)
MAE: 21.677 (0.000)
MAPE: 0.458 (0.000)
R2: 46640.616 (0.000)




### 1.5.3 Train 1 year and predicts one month every year

In [18]:
scores = repeated_blocking_time_series(
    LinearRegression,
    StandardScaler,
    x,
    y,
    train_period=365,
    test_period=30,
    repeats=1,
)
print_scores(scores[0], METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.799 (0.000)
MAE: 0.625 (0.000)
MAPE: 0.013 (0.000)
R2: 0.315 (0.000)


******
[TEST]
******
RMSE: 0.941 (0.000)
MAE: 0.761 (0.000)
MAPE: 0.016 (0.000)
R2: 0.303 (0.000)




### 1.5.4 Train 1 year and 1 month and predicts one month every year

In [19]:
scores = repeated_blocking_time_series(
    LinearRegression,
    StandardScaler,
    x,
    y,
    train_period=395,
    test_period=30,
    repeats=1,
)
print_scores(scores[0], METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.804 (0.000)
MAE: 0.632 (0.000)
MAPE: 0.013 (0.000)
R2: 0.293 (0.000)


******
[TEST]
******
RMSE: 1.162 (0.000)
MAE: 0.796 (0.000)
MAPE: 0.016 (0.000)
R2: 0.110 (0.000)




### 1.5.5 Train 1 year and 2 months and predicts one month every year

In [20]:
scores = repeated_blocking_time_series(
    LinearRegression,
    StandardScaler,
    x,
    y,
    train_period=425,
    test_period=30,
    repeats=1,
)
print_scores(scores[0], METRICS, METRICS_DICT)

******
[TRAIN]
******
RMSE: 0.823 (0.000)
MAE: 0.645 (0.000)
MAPE: 0.014 (0.000)
R2: 0.308 (0.000)


******
[TEST]
******
RMSE: 1.937 (0.000)
MAE: 1.766 (0.000)
MAPE: 0.036 (0.000)
R2: 0.106 (0.000)




In [21]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [22]:
pd.Series(
    pipeline.named_steps["estimator"].coef_, df_no_cs.drop("Date", axis=1).columns
).sort_values().to_frame("Feature Importance").style.background_gradient(cmap="Greens")

,Feature Importance
K2O,-21.434523
Aluminate cubic,-18.755287
Aluminate orto,-13.576397
%Gypsum,-5.899666
SiO2,-4.427702
Fe2O3,-3.659417
Periclase,-3.395747
MgO,-2.623520
Belite alpha,-2.065626
Belite gamma,-1.816506


<IPython.core.display.Javascript object>

In [23]:
pd.Series(
    pipeline.named_steps["estimator"].coef_, df_no_cs.drop("Date", axis=1).columns
).sort_values().to_frame("Feature Importance")

,Feature Importance
K2O,-21.434523
Aluminate cubic,-18.755287
Aluminate orto,-13.576397
%Gypsum,-5.899666
SiO2,-4.427702
Fe2O3,-3.659417
Periclase,-3.395747
MgO,-2.623520
Belite alpha,-2.065626
Belite gamma,-1.816506


<IPython.core.display.Javascript object>

In [24]:
pd.Series(
    pipeline.named_steps["estimator"].coef_, df_no_cs.drop("Date", axis=1).columns
).sort_values()

K2O                    -21.434523
Aluminate cubic        -18.755287
Aluminate orto         -13.576397
%Gypsum                 -5.899666
SiO2                    -4.427702
Fe2O3                   -3.659417
Periclase               -3.395747
MgO                     -2.623520
Belite alpha            -2.065626
Belite gamma            -1.816506
Belite beta             -1.756279
Alite total             -1.416462
Free lime               -0.866655
Dehydration             -0.015054
IR                       0.016382
Total alkali as Na2O     0.090138
LOI                      0.103007
Portlandite              0.120418
Belite total             0.151400
Dolimite                 0.207084
%Limestone               0.748700
Calcite                  1.128016
CaO                      1.215599
Aphthalite               1.498341
SO3                      2.291567
Arcanite                 2.308522
Ferrite                  3.280106
Quartz                   3.442980
Gypsum                   3.809430
Bassanite     

<IPython.core.display.Javascript object>

In [25]:
x

,CaO,MgO,Na2O,Al2O3,SiO2,SO3,K2O,Fe2O3,Total alkali as Na2O,LOI,...,Gypsum,Bassanite,Anhydrite,Calcite,Dolimite,Quartz,%Gypsum,%Limestone,%Clinker,Dehydration
0,60.007980,3.67285,0.19218,4.27263,18.556379,4.43558,1.21612,2.79746,0.92666,4.33,...,0.90,6.24,0.00,4.83,0.04,0.43,7.14,5.420000,86.809998,-100.000000
1,58.848289,2.56056,0.19986,4.18035,17.636169,4.07796,1.20883,3.03309,0.99000,6.24,...,1.15,6.31,0.00,10.29,0.32,0.90,-100.00,-100.000000,-100.000000,-100.000000
2,60.171211,2.42954,0.18575,4.23877,19.038410,3.29525,1.11774,3.24653,1.02000,10.77,...,5.65,1.58,0.00,22.92,0.44,2.10,7.23,25.709999,66.379997,-100.000000
3,60.376080,3.62883,0.19847,4.34362,18.869631,4.15419,1.22138,2.93591,0.93386,4.04,...,1.58,5.53,0.00,5.63,0.24,0.44,7.11,6.420000,85.809998,-100.000000
4,59.927780,2.47666,0.19895,4.33688,19.493290,3.33951,1.10000,3.25291,1.02000,7.78,...,2.68,3.12,0.00,16.17,0.44,1.74,5.80,18.510000,75.050003,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,59.551731,2.20485,0.18921,4.30475,18.163300,4.09706,1.20539,3.03765,0.95054,4.75,...,0.16,4.20,0.20,8.80,1.37,0.98,4.55,11.160000,83.330002,96.570000
590,59.696838,2.16952,0.18454,4.00087,16.895281,3.10755,1.18097,2.67487,0.96679,5.33,...,3.10,1.83,0.01,7.26,1.36,0.93,4.94,9.550000,84.510002,37.230000
591,59.214851,2.28262,0.20063,4.17480,17.821800,4.06848,1.20064,2.95746,0.96439,5.02,...,2.36,2.69,0.00,7.47,1.52,0.78,5.05,9.770000,84.309998,53.259998
592,59.845181,2.69335,0.19684,4.35530,18.473740,4.19836,1.21207,3.02851,0.94394,4.82,...,0.99,3.18,0.04,7.61,1.04,1.06,4.21,9.760000,85.010002,76.440002


<IPython.core.display.Javascript object>

In [26]:
pd.Series(pipeline.named_steps["estimator"].coef_, x.columns).sort_values()

K2O                    -21.434523
Aluminate cubic        -18.755287
Aluminate orto         -13.576397
%Gypsum                 -5.899666
SiO2                    -4.427702
Fe2O3                   -3.659417
Periclase               -3.395747
MgO                     -2.623520
Belite alpha            -2.065626
Belite gamma            -1.816506
Belite beta             -1.756279
Alite total             -1.416462
Free lime               -0.866655
Dehydration             -0.015054
IR                       0.016382
Total alkali as Na2O     0.090138
LOI                      0.103007
Portlandite              0.120418
Belite total             0.151400
Dolimite                 0.207084
%Limestone               0.748700
Calcite                  1.128016
CaO                      1.215599
Aphthalite               1.498341
SO3                      2.291567
Arcanite                 2.308522
Ferrite                  3.280106
Quartz                   3.442980
Gypsum                   3.809430
Bassanite     

<IPython.core.display.Javascript object>

In [27]:
pd.Series(pipeline.named_steps["estimator"].coef_, x.columns).sort_values(
        key=lambda x: abs(x)
    )


Dehydration             -0.015054
IR                       0.016382
Total alkali as Na2O     0.090138
LOI                      0.103007
Portlandite              0.120418
Belite total             0.151400
Dolimite                 0.207084
%Limestone               0.748700
Free lime               -0.866655
Calcite                  1.128016
CaO                      1.215599
Alite total             -1.416462
Aphthalite               1.498341
Belite beta             -1.756279
Belite gamma            -1.816506
Belite alpha            -2.065626
SO3                      2.291567
Arcanite                 2.308522
MgO                     -2.623520
Ferrite                  3.280106
Periclase               -3.395747
Quartz                   3.442980
Fe2O3                   -3.659417
Gypsum                   3.809430
Bassanite                3.888627
SiO2                    -4.427702
%Clinker                 5.291152
%Gypsum                 -5.899666
Anhydrite                9.133105
Na2O          

<IPython.core.display.Javascript object>

In [33]:
pd.Series(
    pipeline.named_steps["estimator"].coef_, x.columns
).sort_values().to_frame().to_csv("Feature_Importance.csv", index=True)

<IPython.core.display.Javascript object>